# ETL Processes
This notebook develops the ETL process for each of your tables before completing the `etl.py` file to load the whole datasets.
After completing 'etl.py', run scripts at the bottom and then check `test.ipynb` to confirm results. 
When this code snippets are rerun, try to execute code snippets related to `conn`,`curr` and `conn.close()` before running `%run ./create_tables.py`. Also, if `test.ipynb` was run, restart `test.ipynb` kernel before rerunning `etl.ipynb`.

In [ ]:
import os
import glob
import psycopg2
import pandas as pd
from sql_queries import *

In [ ]:
%run ./create_tables.py

In [ ]:
conn = psycopg2.connect("host=127.0.0.1 dbname=sparkifydb user=student password=student")
cur = conn.cursor()

# Process `song_data`
In this first part, I will perform ETL on the first dataset, `song_data`, to create the `songs` and `artists` dimensional tables.

I will perform ETL on a single song file and load a single record into each table to start.
- Use the `get_files` function provided above to get a list of all song JSON files in `data/song_data`
- Select the first song in this list
- Read the song file and view the data

In [ ]:
def get_files(filepath):
    """
    navigate through every file in every folder and returns all filepaths as a list
    
    example output:
        ['./data/song_data/A/A/C/TRAACER128F4290F96.json',
         './data/song_data/A/A/C/TRAACPE128F421C1B9.json',
         './data/song_data/A/A/C/TRAACHN128F1489601.json',
         './data/song_data/A/A/C/TRAACOW128F933E35F.json']
    """
    all_files = []
    for root, dirs, files in os.walk(filepath):
        files = glob.glob(os.path.join(root,'*.json'))
        for f in files :
            all_files.append(os.path.abspath(f))
    return all_files

In [ ]:
song_files = get_files('./data/song_data')

In [ ]:
filepath = song_files[0]

In [ ]:
df = pd.read_json(filepath, lines=True)

df.head()

## #1: `songs` Table
#### Extract Data for Songs Table
- Select columns for song ID, title, artist ID, year, and duration
- Use `df.values` to select just the values from the dataframe
- Index to select the first (only) record in the dataframe
- Convert the array to a list and set it to `song_data`

In [ ]:
song_data = df[['song_id','title','artist_id','year','duration']]
song_data = [x for x in song_data.values[0]]
song_data

#### Insert Record into Song Table
Implement the `song_table_insert` query in `sql_queries.py` and run the cell below to insert a record for this song into the `songs` table. Remember to run `create_tables.py` before running the cell below to ensure you've created/resetted the `songs` table in the sparkify database.

In [ ]:
cur.execute(song_table_insert, song_data)
conn.commit()

Run `test.ipynb` to see if you've successfully added a record to this table.

## #2: `artists` Table
#### Extract Data for Artists Table
- Select columns for artist ID, name, location, latitude, and longitude
- Use `df.values` to select just the values from the dataframe
- Index to select the first (only) record in the dataframe
- Convert the array to a list and set it to `artist_data`

In [ ]:
artist_data = df[['artist_id','artist_name','artist_location','artist_latitude','artist_longitude']].values
artist_data = [item for item in artist_data[0]]
artist_data

#### Insert Record into Artist Table
Implement the `artist_table_insert` query in `sql_queries.py` and run the cell below to insert a record for this song's artist into the `artists` table. 

In [ ]:
cur.execute(artist_table_insert, artist_data)
conn.commit()

Run `test.ipynb` to see if you've successfully added a record to this table.

# Process `log_data`
In this part, you'll perform ETL on the second dataset, `log_data`, to create the `time` and `users` dimensional tables, as well as the `songplays` fact table.

Let's perform ETL on a single log file and load a single record into each table.
- Use the `get_files` function provided above to get a list of all log JSON files in `data/log_data`
- Select the first log file in this list
- Read the log file and view the data

In [ ]:
log_files = get_files('./data/log_data')

In [ ]:
filepath = log_files[0]

In [ ]:
df = pd.read_json(filepath, lines=True)
df.head()

## #3: `time` Table
#### Extract Data for Time Table
- Filter records by `NextSong` action
- Convert the `ts` timestamp column to datetime
- Extract the timestamp, hour, day, week of year, month, year, and weekday from the `ts` column and set `time_data` to a list containing these values in order
- Specify labels for these columns and set to `column_labels`
- Create a dataframe, `time_df,` containing the time data for this file by combining `column_labels` and `time_data` into a dictionary and converting this into a dataframe

In [ ]:
df = df[df['page']=='NextSong']
df.head()

In [ ]:
t = pd.to_datetime(df['ts'],unit='ms')
t.head()

In [ ]:
# timestamp, hour, day, week of year, month, year, and weekday
column_labels =['start_time', 'hour', 'day', 'week', 'month', 'year', 'weekday'] 
time_data = (df['ts'].astype(str) +','+ t.dt.strftime('%H, %d, %W, %m, %Y, %a')).str.split(',')
time_data.head()

In [ ]:
time_df = pd.DataFrame.from_dict(dict(time_data)).T
time_df.columns = column_labels
time_df.loc[:,'start_time']=pd.to_datetime(time_df['start_time'],unit='ms').astype(int)
time_df.loc[:,'hour']=time_df['hour'].astype(int)
time_df.loc[:,'day']=time_df['day'].astype(int)
time_df.loc[:,'week']=time_df['week'].astype(int)
time_df.loc[:,'month']=time_df['month'].astype(int)
time_df.loc[:,'year']=time_df['year'].astype(int)
time_df.loc[:,'weekday']=time_df['weekday'].str.strip()
time_df.info()

#### Insert Records into Time Table
Implement the `time_table_insert` query in `sql_queries.py` and run the cell below to insert records for the timestamps in this log file into the `time` table.

In [ ]:
for i, row in time_df.iterrows():
    cur.execute(time_table_insert, list(row))
    conn.commit()

Run `test.ipynb` to see if you've successfully added records to this table.

## #4: `users` Table
#### Extract Data for Users Table
- Select columns for user ID, first name, last name, gender and level and set to `user_df`

In [ ]:
user_df = df[['userId','firstName','lastName','gender','level']]

#### Insert Records into Users Table
Implement the `user_table_insert` query in `sql_queries.py` and run the cell below to insert records for the users in this log file into the `users` table. 

In [ ]:
for i, row in user_df.iterrows():
    cur.execute(user_table_insert, row)
    conn.commit()

Run `test.ipynb` to see if you've successfully added records to this table.

## #5: `songplays` Table
#### Extract Data and Songplays Table

- Implement the `song_select` query in `sql_queries.py` to find the song ID and artist ID based on the title, artist name, and duration of a song.
- Select the timestamp, user ID, level, song ID, artist ID, session ID, location, and user agent and set to `songplay_data`

#### Insert Records into Songplays Table
- Implement the `songplay_table_insert` query and run the cell below to insert records for the songplay actions in this log file into the `songplays` table. Remember to run `create_tables.py` before running the cell below to ensure you've created/resetted the `songplays` table in the sparkify database.

In [ ]:
songplay_df = df
songplay_df['ts'] = pd.to_datetime(songplay_df['ts'],unit='ms').astype(int)

songplay_df.info()

In [ ]:
for index, row in df.iterrows():
    
    # get songid and artistid from song and artist tables
    cur.execute(song_select, (row.song, row.artist, row.length))
    results = cur.fetchone()
    
    if results:
        songid, artistid = results
    else:
        songid, artistid = None, None

    # insert songplay record
    songplay_data = [index] + row[['ts', 'userId', 'level','sessionId','location','userAgent']].tolist() + [songid, artistid]
    cur.execute(songplay_table_insert, songplay_data)
    conn.commit()

Run `test.ipynb` to see if you've successfully added records to this table.

# Close Connection to Sparkify Database

In [ ]:
conn.close()

# Implement `etl.py`
In order to see how `etl.py` works, restart the kernel and run following code snippets.
Then, confirm results using `test.ipynb`

In [ ]:
import os
import glob
import psycopg2
import pandas as pd
from sql_queries import *

%run ./create_tables.py
%run ./etl.py